In [21]:
import ipywidgets as widgets
import pandas as pd

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue() #tell the thread when to stop recording
recordings = Queue() #store the recordings


transcriptions = []
transcribe = None


sia = SentimentIntensityAnalyzer()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style = 'success',
    icon = 'microphone'
)




stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style = 'warning',
    icon = 'pause'
)


def start_recording(data):
    messages.put(True)
    
    with output:
        display("Recording...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
        
def stop_recording(data):
    global transcriptions, transcribe
    with output:
        messages.get()
        display("Stopped recording")
        
        # Wait for the transcription thread to finish
        if transcribe is not None:
            transcribe.join()
            

def analyze_sentiment(data):
    # Create a DataFrame from all transcriptions and save to CSV
    if transcriptions:  # Make sure the list is not empty
        df = pd.DataFrame(transcriptions, columns=['TEXT'])
        df['Negative'], df['Neutral'], df['Positive'], df['Emotion'], df['Subjectivity'], df['Opinion/Fact?'], df['Weighted Sentiment'] = zip(*df['TEXT'].map(getSentiment))
        df.to_csv('all_transcriptions.csv', index=False)
        
        # Style DataFrame
        styled = df.style.set_table_styles([
            {"selector": "th", "props": [("background-color", "white")]},
            {"selector": "td", "props": [("background-color", "white")]}
        ])

        display(styled)
    else:
        display("No transcriptions to analyze")
        

analyze_button = widgets.Button(
    description="Analyze",
    disabled=False,
    button_style = 'danger',
    icon = 'stop'
)
    


#Click xong thì nó sẽ gọi hàm start_recording và stop_recording
record_button.on_click(start_recording) 
stop_button.on_click(stop_recording)    
analyze_button.on_click(analyze_sentiment)

output = widgets.Output()
display(record_button, stop_button, analyze_button, output)    

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='pause', style=ButtonStyle())

Button(button_style='danger', description='Analyze', icon='stop', style=ButtonStyle())

Output()

In [22]:
import ipywidgets as widgets
import pandas as pd

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue() #tell the thread when to stop recording
recordings = Queue() #store the recordings


transcriptions = []
transcribe = None


sia = SentimentIntensityAnalyzer()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style = 'success',
    icon = 'microphone'
)




stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style = 'warning',
    icon = 'pause'
)


def start_recording(data):
    messages.put(True)
    
    with output:
        display("Recording...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
        
def stop_recording(data):
    global transcriptions, transcribe
    with output:
        messages.get()
        display("Stopped recording")
        
        # Wait for the transcription thread to finish
        if transcribe is not None:
            transcribe.join()
            

def analyze_sentiment(data):
    # Create a DataFrame from all transcriptions and save to CSV
    if transcriptions:  # Make sure the list is not empty
        df = pd.DataFrame(transcriptions, columns=['TEXT'])
        df['Negative'], df['Neutral'], df['Positive'], df['Emotion'], df['Subjectivity'], df['Opinion/Fact?'], df['Weighted Sentiment'] = zip(*df['TEXT'].map(getSentiment))
        df.to_csv('all_transcriptions.csv', index=False)
        
        # Style DataFrame
        styled = df.style.set_table_styles([
            {"selector": "th", "props": [("background-color", "white")]},
            {"selector": "td", "props": [("background-color", "white")]}
        ])

        display(styled)
    else:
        display("No transcriptions to analyze")
        

analyze_button = widgets.Button(
    description="Analyze",
    disabled=False,
    button_style = 'danger',
    icon = 'stop'
)
    


#Click xong thì nó sẽ gọi hàm start_recording và stop_recording
record_button.on_click(start_recording) 
stop_button.on_click(stop_recording)    
analyze_button.on_click(analyze_sentiment)

output = widgets.Output()
display(record_button, stop_button, analyze_button, output)    

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='pause', style=ButtonStyle())

Button(button_style='danger', description='Analyze', icon='stop', style=ButtonStyle())

Output()

,TEXT,Negative,Neutral,Positive,Emotion,Subjectivity,Opinion/Fact?,Weighted Sentiment
0,i had three house,0.000000,1.000000,0.000000,Neutral 😐,0.000000,0,0.000000


In [23]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (4- Realtek(R)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'FxSound Speakers (FxSound Audio', 'hostApi': 0, 'maxInputChann

In [24]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 5
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk = 1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format=AUDIO_FORMAT, channels=CHANNELS, rate=FRAME_RATE, input=True,input_device_index=1, frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        #every record_seconds seconds, put the frames into the queue, then start recording another record_seconds audio 
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS/chunk):
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()
            

In [25]:
import json

from vosk import Model, KaldiRecognizer

model = Model(r'E:\Homework\AI\vosk-model-small-en-us-0.15\vosk-model-small-en-us-0.15')
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    global transcriptions  # Declare transcriptions as global
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)['text']
        output.append_stdout(text)
        transcriptions.append(text)  # Save transcription
        

In [26]:

def getSentiment(text):
    sentiment = sia.polarity_scores(text)
    blob = TextBlob(text)
    subjectivity = blob.sentiment.subjectivity

    opinion_fact = int(subjectivity > 0.5) 
    # Assuming subjectivity score > 0.5 indicates an opinion
    if opinion_fact == 1:
        opinion_fact_text = 'Opinion'
    else:
        opinion_fact = 'Fact'
    weighted_sentiment_text = sentiment['compound'] * subjectivity
    return sentiment['neg'], sentiment['neu'], sentiment['pos'], getSentimentLabel(sentiment), subjectivity, opinion_fact_text, weighted_sentiment

def getSentimentLabel(sentiment):
    if sentiment['compound'] >= 0.05:
        return 'Positive 😃'
    elif sentiment['compound'] <= -0.05:
        return 'Negative 😞'
    else:
        return 'Neutral 😐'
    



In [27]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (4- Realtek(R)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'FxSound Speakers (FxSound Audio', 'hostApi': 0, 'maxInputChann

In [28]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 5
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk = 1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format=AUDIO_FORMAT, channels=CHANNELS, rate=FRAME_RATE, input=True,input_device_index=1, frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        #every record_seconds seconds, put the frames into the queue, then start recording another record_seconds audio 
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS/chunk):
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()
            

In [29]:
import json

from vosk import Model, KaldiRecognizer

model = Model(r'E:\Homework\AI\vosk-model-small-en-us-0.15\vosk-model-small-en-us-0.15')
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
    global transcriptions  # Declare transcriptions as global
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)['text']
        output.append_stdout(text)
        transcriptions.append(text)  # Save transcription
        

In [30]:

def getSentiment(text):
    sentiment = sia.polarity_scores(text)
    blob = TextBlob(text)
    subjectivity = blob.sentiment.subjectivity
    opinion_fact = int(subjectivity > 0.5)  # Assuming subjectivity score > 0.5 indicates an opinion
    if opinion_fact == 1:
        opinion_fact = 'Opinion'
    else:
        opinion_fact = 'Fact'
    weighted_sentiment = sentiment['compound'] * subjectivity
    return sentiment['neg'], sentiment['neu'], sentiment['pos'], getSentimentLabel(sentiment), subjectivity, opinion_fact, weighted_sentiment

def getSentimentLabel(sentiment):
    if sentiment['compound'] >= 0.05:
        return 'Positive 😃'
    elif sentiment['compound'] <= -0.05:
        return 'Negative 😞'
    else:
        return 'Neutral 😐'
    

